In [1]:
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform

from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('dataset2_1.csv')
columns = [column for column in df.columns if column not in ['image', 'label']]
x = df[columns].to_numpy()
y = df['label'].to_numpy()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8)

model = make_pipeline(StandardScaler(), SVR(C=35, epsilon=0.1))
model.fit(x_train, y_train)
print("Accuracy on test data: {:.2f}".format(model.score(x_test, y_test)))

Accuracy on test data: 0.25


In [3]:
result = permutation_importance(model, x_train, y_train, n_repeats=90, random_state=42)
perm_sorted_idx = result.importances_mean.argsort()

In [4]:
perm_sorted_idx

array([31, 11, 23,  6,  5, 16,  8,  3, 27,  2, 21, 24, 28,  7,  0, 30, 13,
       19, 29, 26,  9, 17, 10, 25, 14,  4,  1, 15, 18, 20, 22, 12],
      dtype=int64)

In [5]:
labels=list()
for idx in perm_sorted_idx:
    labels.append(columns[idx])

In [6]:
labels

['f16_mac',
 'f12_mic',
 'f8_mac',
 'f7_mic',
 'f6_mic',
 'f1_mac',
 'f9_mic',
 'f4_mic',
 'f12_mac',
 'f3_mic',
 'f6_mac',
 'f9_mac',
 'f13_mac',
 'f8_mic',
 'f1_mic',
 'f15_mac',
 'f14_mic',
 'f4_mac',
 'f14_mac',
 'f11_mac',
 'f10_mic',
 'f2_mac',
 'f11_mic',
 'f10_mac',
 'f15_mic',
 'f5_mic',
 'f2_mic',
 'f16_mic',
 'f3_mac',
 'f5_mac',
 'f7_mac',
 'f13_mic']

In [13]:
fig, ax = plt.subplots(1, 1, figsize=(80, 240))
ax.boxplot(
    result.importances[perm_sorted_idx].T,
    vert=False,
    labels=labels,
)
fig.tight_layout()
plt.rcParams.update({'font.size': 22})
plt.show()
plt.savefig('importances_mean_corr_1.jpg')

<Figure size 432x288 with 0 Axes>

In [12]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(240, 160))
corr = spearmanr(x).correlation

corr = (corr + corr.T) / 2
np.fill_diagonal(corr, 1)

distance_matrix = 1 - np.abs(corr)
dist_linkage = hierarchy.ward(squareform(distance_matrix))
dendro = hierarchy.dendrogram(
    dist_linkage, labels=columns, ax=ax1, leaf_rotation=90
)
dendro_idx = np.arange(0, len(dendro["ivl"]))

ax2.imshow(corr[dendro["leaves"], :][:, dendro["leaves"]])
ax2.set_xticks(dendro_idx)
ax2.set_yticks(dendro_idx)
ax2.set_xticklabels(dendro["ivl"], rotation="vertical")
ax2.set_yticklabels(dendro["ivl"])
fig.tight_layout()
plt.rcParams.update({'font.size': 22})
plt.show()
plt.savefig('correlation_1.png')

<Figure size 432x288 with 0 Axes>

In [9]:
cluster_ids = hierarchy.fcluster(dist_linkage, 1, criterion="distance")
cluster_id_to_feature_ids = defaultdict(list)
for idx, cluster_id in enumerate(cluster_ids):
    cluster_id_to_feature_ids[cluster_id].append(idx)
selected_features = [v[0] for v in cluster_id_to_feature_ids.values()]

X_train_sel = x_train[:, selected_features]
X_test_sel = x_test[:, selected_features]

model = make_pipeline(StandardScaler(), SVR(C=35, epsilon=0.1))
model.fit(X_train_sel, y_train)
print("Accuracy on test data: {:.2f}".format(model.score(X_test_sel, y_test)))

Accuracy on test data: 0.24


In [10]:
len(selected_features)

7

In [14]:
print(selected_features)

[0, 1, 2, 3, 4, 8, 9]


In [15]:
cluster_id_to_feature_ids

defaultdict(list,
            {7: [0, 6, 24, 28, 29],
             1: [1, 7, 12, 15, 19],
             6: [2, 18, 26, 27, 31],
             3: [3, 5, 11, 16, 20, 30],
             5: [4, 13, 22, 25],
             4: [8, 10, 17, 21],
             2: [9, 14, 23]})